## Import python packages

In [4]:
from osgeo import gdal
import numpy as np
import os

In [5]:
os.getcwd()
os.chdir('E:/W_Katrin/Kigali/Raster/')
os.getcwd()

'E:\\W_Katrin\\Kigali\\Raster'

In [6]:
file_path = r"2015_Pleiades_Kigali_subset.tif"
single_band = True # set single band true to calculate for each band a percentile. False --> calculate one percentiule for all bands 
percentile = (0.02,0.98)# set the percentile 

In [11]:
def get_max_min_by_bands(file_path, percentile, single_band=True):
    image_raw = gdal.Open(file_path) # opens the image 
    img_array = image_raw.ReadAsArray()  # reads image into array missing values get the value 0
    perc_min, perc_max = percentile  # reads the the percentile into perc_min, perc_max
    bands = img_array.shape[0] # reads out the number of bands
    bands = 3 # if RGB wanted
    img_perc = []   
    if single_band == True:  # 
        for b in range(bands):
            band_array = img_array[b]
    
            resh_band_array = np.reshape(band_array,(-1,1))
            img_squeez = np.squeeze(resh_band_array, axis=1)
            print(img_squeez.shape)
            img_squeez = np.delete(img_squeez, img_squeez == 0)
            print(img_squeez.shape)
            img_size = img_squeez.shape[0]
            img_sort_listed = np.sort(img_squeez)
        
            img_max_var= int(img_size* perc_max-1)
            img_min_var= int(img_size* perc_min)
     
            img_max = img_sort_listed[img_max_var]
            img_min = img_sort_listed[img_min_var]
        
            img_perc.append(img_min)
            img_perc.append(img_max)
    if single_band == False:
        band_array = img_array
        resh_band_array = np.reshape(band_array,(-1,1))
        img_squeez = np.squeeze(resh_band_array, axis=1)
        img_size = img_squeez.shape[0]
        img_sort_listed = np.sort(img_squeez)
        
        img_max_var= int(img_size* perc_max-1)
        img_min_var= int(img_size* perc_min)
     
        img_max = img_sort_listed[img_max_var]
        img_min = img_sort_listed[img_min_var]
        
        img_perc.append(img_min)
        img_perc.append(img_max)
    return img_perc

In [12]:
max_min = get_max_min_by_bands(file_path, percentile, single_band)

(108810254,)
(108810254,)
(108810254,)
(108810254,)
(108810254,)
(108810254,)


In [13]:
print(max_min)

[326.29517, 1051.6909, 391.3888, 875.3834, 447.24457, 854.81647]


In [14]:
def conv_img(img, imin, imax, target_size, target_type):
    
    target_min, target_max = target_size #reads out min and max in target_size
    a = (target_max - target_min) / (imax -imin) # builds the rescale steps 
    img = np.where(img == 0, imin, img)
    b = img - imin
    new_img = (a * b)
    new_img = np.where(new_img>255, 255, new_img)
    new_img = np.where(new_img<0, 0, new_img)
    new_img.astype(target_type)
    return new_img

In [15]:
def converter(img_path, output_path, target_size, img_perc, single_band):
    # loading in the input img 
    img = gdal.Open(img_path)
    img_array = img.ReadAsArray()
    bands = img_array.shape[0]
    bands = 3 # if RGB wanted
    width = img.RasterXSize
    height = img.RasterYSize
    format = "GTiff"
    driver = gdal.GetDriverByName(format)
    target_type = np.uint8
    dst_ds = driver.Create(output_path, width, height, bands)
    
    width = img.RasterXSize
    height = img.RasterYSize
    if single_band == True:
        for b in range (bands):
            imin = img_perc[b*2]
            imax = img_perc[b*2+1]
            raster_array = img.GetRasterBand(b+1).ReadAsArray()
            raster_conv = conv_img(raster_array, imin, imax, target_size, target_type)
            dst_ds.GetRasterBand(b+1).WriteArray(raster_conv)
    if single_band == False:
        for b in range (bands):
            imin = img_perc[0]
            imax = img_perc[1]
            raster_array = img.GetRasterBand(b+1).ReadAsArray()
            raster_conv = conv_img(raster_array, imin, imax, target_size, target_type)
            dst_ds.GetRasterBand(b+1).WriteArray(raster_conv)

In [16]:
img_path = r"2015_Pleiades_Kigali_subset.tif"
output_path = r'2015_Pleiades_Kigali_subset_.tif'
target_size = (0,255)
max_min = max_min
single_band = True 

In [17]:
converter(img_path, output_path, target_size, max_min, single_band)

In [18]:
target_min, target_max = target_size #reads out min and max in target_size
imin = max_min[2]
print("imin is: ", imin)
imax = max_min[3]
print("imax is: ", imax)
img = 0.263478
a = (target_max - target_min) / (imax - imin) # builds the rescale steps 
print("a is: ", a)
b = img - imin
print("b is: ", b)
new_img = a*b
new_img = np.where(new_img>255, 255, new_img)
new_img = np.where(new_img<0, 0, new_img)
print(new_img)

imin is:  391.3888
imax is:  875.3834
a is:  0.5268653509156889
b is:  -391.1253159453125
0.0
